 <a id="Title"></a>
# <div style="width:auto;text-align:center; border-radius:5px 5px; padding:10px; color:white; margin: 20px auto; font-size:90%; font-family:Pacifico; background-color:#b825f7; overflow:hidden;color:white;border:2px solid #b825f7"><b>Automated Job Scraper</b></div>

In [1]:
!pip install selenium

  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
Using cached selenium-4.35.0-py3-none-any.whl (9.6 MB)
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains


 <a id="Title"></a>
# <div style="width:900px;text-align:center; border-radius:5px 5px; padding:10px; color:white; margin: 20px auto; font-size:90%; font-family:Pacifico; background-color:#b825f7; overflow:hidden;color:white;border:2px solid #b825f7"><b>Script</b></div>

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu") # Disables GPU hardware acceleration.
chrome_options.add_argument("--disable-extensions") # Disables installed extensions.

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get("https://www.stepstone.de/jobs/ingenieur-in/in-stuttgart?radius=50")
cookie_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "ccmgt_explicit_accept"))
    )

wait=WebDriverWait(driver,15)
def wait_for_page_to_load(driver,wait):
# wait for the webpage to load
    page_title=driver.title
    try:
        wait.until(

            lambda d: d.execute_script("return document.ready_state") == "complete"
        )
    except:
        print(f"The page \"{page_title}\" did not get fully loaded within the given duration")
    else:
        print(f"The page \"{page_title}\" is successfully loaded")

cookie_button.click()
wait_for_page_to_load(driver,wait)

title=[]
url=[]
company_name=[]
location=[]
posted_date=[]
description=[]
listing_id=[]
try:
    while True:
        wait_for_page_to_load(driver,wait)
        ele=driver.find_element(By.XPATH,'//*[@id="app-unifiedResultlist"]/div/div[3]/div/div[2]/div[2]/div')
        all_ele=ele.find_elements(By.CSS_SELECTOR, "article")
        for ele in all_ele:
            # Title
            try:
                title.append(ele.find_element(By.CSS_SELECTOR, 'h2').text)
            except:
                title.append("Unknown")

            # URL
            try:
                uri = ele.find_element(By.CSS_SELECTOR, 'h2 a').get_attribute('href')
                url.append(uri)
                list_id = uri.split('--')[-1].split('-')[0]
                listing_id.append(list_id)
            except:
                url.append("Unknown")
                listing_id.append("Unknown")

            # Company Name
            try:
                company = ele.find_element(By.XPATH, './/div[1]/div[2]/div[1]/div/span/span/span[2]').text
                company_name.append(company)
            except:
                company_name.append("Unknown")

            # Location
            try:
                loc = ele.find_element(By.XPATH, './/div[1]/div[2]/div[2]/span/span[2]').text
                location.append(loc)
            except:
                location.append("Unknown")


            # Posted Date
            try:
                pd = ele.find_element(By.XPATH, './/div[2]/div[3]/div[1]/span').text.strip()

                # If it's empty, try the second one
                if not pd:
                    pd = ele.find_element(By.XPATH, './/div[2]/div[2]/div[1]/span').text.strip()

                # Still empty? Set to Unknown
                if not pd:
                    pd = "Unknown"
                posted_date.append(pd)

            except:
                posted_date.append("Unknown")

            # Description
            try:
                desc = ele.find_element(By.XPATH, './/div[2]/div[1]/div/span/div/div/span').text.strip()

                # If it's empty, try the second one
                if not desc:
                    desc = ele.find_element(By.XPATH, './/div[2]/div[2]/div/span/div/div/span').text.strip()

                if not desc:
                    desc = "Unknown"
            except:
                try:
                    desc = ele.find_element(By.XPATH, './/div[2]/div[2]/div/span/div/div/span').text.strip()
                    if not desc:
                        desc = "Unknown"
                except:
                    desc = "Unknown"

            description.append(desc)


        try :
                # Wait for the obstructing element to be invisible or stale
                wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '.js-japubox-popover-modal')))
                next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[aria-label="Nächste"]')))
                ActionChains(driver).move_to_element(next_button).perform()
        except:
                print("\n next button is not clickable or the pop-up is persistent")
                break

        else:
                next_button.click()
                time.sleep(1)
finally:
    driver.quit()

The page "Ingenieur In: 2.062 Jobs & Stellenangebote in Stuttgart | 1. Oct 2025" did not get fully loaded within the given duration
The page "Ingenieur In: 2.062 Jobs & Stellenangebote in Stuttgart | 1. Oct 2025" did not get fully loaded within the given duration

 next button is not clickable or the pop-up is persistent


In [12]:
display(df['company_name'].unique())

array(['Unknown'], dtype=object)

In [13]:
import pandas as pd
df=pd.DataFrame({
    'title':title,
    'url':url,
    'company_name':company_name,
    'location':location,
    'posted_date':posted_date,
    'description':description,
    'listing_id':listing_id
})
display(df.shape)

(27, 7)

In [14]:
display(df.shape)

(27, 7)

In [15]:
display(df.head())

,title,url,company_name,location,posted_date,description,listing_id
0,Ingenieur / Techniker (m/w/d) OT-Netze / IP-Netze,https://www.stepstone.de/stellenangebote--Inge...,Unknown,Stuttgart,vor 23 Stunden,Unknown,13014901
1,Ingenieur als Planer von Energiezentralen (m/w/d),https://www.stepstone.de/stellenangebote--Inge...,Unknown,Stuttgart,vor 1 Tag,Unknown,13012741
2,Ingenieur / Techniker (w/m/d) Objektüberwachun...,https://www.stepstone.de/stellenangebote--Inge...,Unknown,"Stuttgart, Mannheim, Frankfurt, Köln, Dresden,...",vor 1 Tag,Unknown,12625448
3,Ingenieur Assetmanagement (w/m/d),https://www.stepstone.de/stellenangebote--Inge...,Unknown,Stuttgart,vor 6 Tagen,Unknown,13054300
4,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown


In [16]:
import pandas as pd
df=pd.DataFrame({
    'title':title,
    'url':url,
    'company_name':company_name,
    'location':location,
    'posted_date':posted_date,
    'description':description,
    'listing_id':listing_id
})

In [17]:
df.shape

(27, 7)

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df = df[~(df == "Unknown").all(axis=1)]

In [20]:
(df['description']=='Unknown').sum()

np.int64(25)

In [21]:
df.tail()

,title,url,company_name,location,posted_date,description,listing_id
22,Techniker / Ingenieur Reinraumtechnik (m/w/d),https://www.stepstone.de/stellenangebote--Tech...,Unknown,Stuttgart,vor 4 Wochen,Unknown,11893278
23,Senior Bauingenieur:in Brückenbau (m/w/d),https://www.stepstone.de/stellenangebote--Seni...,Unknown,"Offenburg, Stuttgart",vor 2 Wochen,Unknown,12616244
24,Entwicklungsingenieur Hardware-in-the-Loop (m/...,https://www.stepstone.de/stellenangebote--Entw...,Unknown,Stuttgart,vor 3 Wochen,Unknown,7904711
25,Ingenieurin / Ingenieur (m/w/d) (FH-Diplom / B...,https://www.stepstone.de/stellenangebote--Inge...,Unknown,Lauffen am Neckar,vor 6 Tagen,Unknown,13054431
26,Applikationsingenieur:in (all genders) Industr...,https://www.stepstone.de/stellenangebote--Appl...,Unknown,Ludwigsburg,vor 1 Tag,Unknown,12943592


In [22]:
 df.to_csv("stepstone_jobs_stuggi.csv", index=False)